In [1]:
import pandas as pd
import numpy as np

from sklearn.preprocessing import StandardScaler
from sklearn.feature_selection import SelectFromModel
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import cross_val_score
from lightgbm import LGBMClassifier
from sklearn.metrics import log_loss

In [12]:
X_train = pd.read_csv('features_tr.csv',encoding='cp949')
X_test = pd.read_csv('features_te.csv', encoding='cp949')
y_train = pd.read_csv('y_train.csv', encoding='cp949')

In [13]:
X_train

,child_num,income_total,DAYS_BIRTH,DAYS_EMPLOYED,FLAG_MOBIL,work_phone,phone,email,family_size,begin_month,...,occyp_type_Low-skill Laborers,occyp_type_Managers,occyp_type_Medicine staff,occyp_type_Private service staff,occyp_type_Realty agents,occyp_type_Sales staff,occyp_type_Secretaries,occyp_type_Security staff,occyp_type_Waiters/barmen staff,occyp_type_no job
0,0,202500.0,-13899,-4709,1,0,0,0,2.0,6.0,...,0,0,0,0,0,0,0,0,0,1
1,1,247500.0,-11380,-1540,1,0,0,1,3.0,5.0,...,0,0,0,0,0,0,0,0,0,0
2,0,450000.0,-19087,-4434,1,0,1,0,2.0,22.0,...,0,1,0,0,0,0,0,0,0,0
3,0,202500.0,-15088,-2092,1,0,1,0,2.0,37.0,...,0,0,0,0,0,1,0,0,0,0
4,0,157500.0,-15037,-2105,1,0,0,0,2.0,26.0,...,0,1,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
26452,2,225000.0,-12079,-1984,1,0,0,0,4.0,2.0,...,0,0,0,0,0,0,0,0,0,0
26453,1,180000.0,-15291,-2475,1,0,0,0,2.0,47.0,...,0,0,0,0,0,0,0,0,0,1
26454,0,292500.0,-10082,-2015,1,0,0,0,2.0,25.0,...,0,0,0,0,0,0,0,0,0,0
26455,0,171000.0,-10145,-107,1,0,0,0,1.0,59.0,...,0,0,0,0,0,0,0,0,0,0


In [3]:
print(X_train.shape, X_test.shape, y_train.shape)

(26457, 62) (10000, 62) (26457, 1)


In [9]:
X_test

,child_num,income_total,DAYS_BIRTH,DAYS_EMPLOYED,FLAG_MOBIL,work_phone,phone,email,family_size,begin_month,...,occyp_type_Low-skill Laborers,occyp_type_Managers,occyp_type_Medicine staff,occyp_type_Private service staff,occyp_type_Realty agents,occyp_type_Sales staff,occyp_type_Secretaries,occyp_type_Security staff,occyp_type_Waiters/barmen staff,occyp_type_no job
0,0,112500.0,-21990,0,1,0,1,0,2.0,60.0,...,0,0,0,0,0,0,0,0,0,1
1,0,135000.0,-18964,-8671,1,0,1,0,2.0,36.0,...,0,0,0,0,0,0,0,0,0,0
2,0,69372.0,-15887,-217,1,1,1,0,2.0,40.0,...,0,0,0,0,0,0,0,0,0,0
3,0,112500.0,-19270,-2531,1,1,0,0,2.0,41.0,...,0,0,0,0,0,0,0,0,0,0
4,0,225000.0,-17822,-9385,1,1,0,0,2.0,8.0,...,0,1,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,0,202500.0,-18593,-5434,1,1,1,0,2.0,19.0,...,0,0,0,0,0,0,0,0,0,0
9996,0,202500.0,-10886,-1315,1,1,0,0,2.0,34.0,...,0,0,0,0,0,0,0,0,0,0
9997,0,292500.0,-21016,-14018,1,0,0,0,2.0,55.0,...,0,0,1,0,0,0,0,0,0,0
9998,0,180000.0,-16541,-1085,1,0,1,0,2.0,33.0,...,0,0,0,0,0,0,0,0,0,1


## tuning

In [4]:
import optuna
def objective_lgbm(trial):
    lgbm_random_state = 0
    lgbm_n_jobs = -1
    lgbm_num_leaves = trial.suggest_int("num_leaves", 50, 80)
    lgbm_min_child_samples = trial.suggest_int('min_child_samples',50,100)
    lgbm_max_depth = trial.suggest_int('max_depth',5,60)
    lgbm_learning_rate = trial.suggest_float('learning_rate',0.0001,0.7)
    lgbm_n_estimators = trial.suggest_int('n_estimators',50,500)
    lgbm_reg_lambda = trial.suggest_float('reg_lambda',0,1)
    lgbm_reg_alpha = trial.suggest_float('reg_alpha',0,1)
    
    
    
    
    classifier_obj = LGBMClassifier(num_leaves = lgbm_num_leaves,
                                    random_state = 0,
                                    n_jobs = lgbm_n_jobs,
                                    min_child_samples = lgbm_min_child_samples,
                                    max_depth = lgbm_max_depth,
                                    learning_rate = lgbm_learning_rate,
                                    n_estimators = lgbm_n_estimators,
                                    reg_lambda = lgbm_reg_lambda,
                                    reg_alpha = lgbm_reg_alpha,
                                    devie='gpu'
                                   )
    
    score = cross_val_score(classifier_obj, X_train,  y_train, scoring='neg_log_loss', cv=5, n_jobs=-1)
    logloss = score.mean()
    return logloss

study_lgbm = optuna.create_study(direction="maximize")
study_lgbm.optimize(objective_lgbm, n_trials=50)
print(study_lgbm.best_trial.value,study_lgbm.best_trial.params)

[I 2022-11-27 14:40:32,635] A new study created in memory with name: no-name-67244530-81b5-4c42-9f1a-9a94d8340c48
[I 2022-11-27 14:40:38,751] Trial 0 finished with value: -1.1265722960320044 and parameters: {'num_leaves': 63, 'min_child_samples': 67, 'max_depth': 11, 'learning_rate': 0.6645175417917235, 'n_estimators': 341, 'reg_lambda': 0.19638725091069853, 'reg_alpha': 0.36696960593193284}. Best is trial 0 with value: -1.1265722960320044.
[I 2022-11-27 14:40:40,974] Trial 1 finished with value: -0.8292571960278969 and parameters: {'num_leaves': 66, 'min_child_samples': 75, 'max_depth': 57, 'learning_rate': 0.6024040786606151, 'n_estimators': 102, 'reg_lambda': 0.20509172707918166, 'reg_alpha': 0.8722758578936417}. Best is trial 1 with value: -0.8292571960278969.
[I 2022-11-27 14:40:43,254] Trial 2 finished with value: -0.8588497081697838 and parameters: {'num_leaves': 75, 'min_child_samples': 51, 'max_depth': 39, 'learning_rate': 0.6322871634935419, 'n_estimators': 99, 'reg_lambda': 

[I 2022-11-27 14:41:29,167] Trial 24 finished with value: -0.8178941873277841 and parameters: {'num_leaves': 80, 'min_child_samples': 93, 'max_depth': 31, 'learning_rate': 0.002547292484462775, 'n_estimators': 250, 'reg_lambda': 0.44570640133917455, 'reg_alpha': 0.7706669923074085}. Best is trial 22 with value: -0.7347345617957105.
[I 2022-11-27 14:41:31,015] Trial 25 finished with value: -0.7398357701042078 and parameters: {'num_leaves': 77, 'min_child_samples': 95, 'max_depth': 45, 'learning_rate': 0.060526404018030396, 'n_estimators': 169, 'reg_lambda': 0.5879525944669332, 'reg_alpha': 0.829072215345306}. Best is trial 22 with value: -0.7347345617957105.
[I 2022-11-27 14:41:34,767] Trial 26 finished with value: -0.7466810003487966 and parameters: {'num_leaves': 77, 'min_child_samples': 96, 'max_depth': 26, 'learning_rate': 0.07589301873801307, 'n_estimators': 366, 'reg_lambda': 0.27352331487136394, 'reg_alpha': 0.6956956544556532}. Best is trial 22 with value: -0.7347345617957105.
[

[I 2022-11-27 14:42:32,453] Trial 48 finished with value: -0.7678872220880477 and parameters: {'num_leaves': 69, 'min_child_samples': 92, 'max_depth': 52, 'learning_rate': 0.10285629391861686, 'n_estimators': 458, 'reg_lambda': 0.6783499396625732, 'reg_alpha': 0.7504524105013158}. Best is trial 22 with value: -0.7347345617957105.
[I 2022-11-27 14:42:33,429] Trial 49 finished with value: -0.7497656741846617 and parameters: {'num_leaves': 74, 'min_child_samples': 77, 'max_depth': 48, 'learning_rate': 0.2539163476870553, 'n_estimators': 95, 'reg_lambda': 0.8193694245816103, 'reg_alpha': 0.5109200764749172}. Best is trial 22 with value: -0.7347345617957105.


-0.7347345617957105 {'num_leaves': 77, 'min_child_samples': 75, 'max_depth': 24, 'learning_rate': 0.05962509116013047, 'n_estimators': 234, 'reg_lambda': 0.30710121460984924, 'reg_alpha': 0.8263393436726127}


-0.7347345617957105 {'num_leaves': 77, 'min_child_samples': 75, 'max_depth': 24, 'learning_rate': 0.05962509116013047, 'n_estimators': 234, 'reg_lambda': 0.30710121460984924, 'reg_alpha': 0.8263393436726127}

In [6]:
lgbm_params =  {'num_leaves': 77, 'min_child_samples': 75, 'max_depth': 24, 'learning_rate': 0.05962509116013047, 'n_estimators': 234, 'reg_lambda': 0.30710121460984924, 'reg_alpha': 0.8263393436726127}

## training

In [7]:
train_org = pd.read_csv('../open/train.csv', encoding='cp949')

In [8]:
skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
folds=[]
for train_idx, valid_idx in skf.split(train_org, train_org['credit']):
    folds.append((train_idx, valid_idx))

In [9]:
import random

In [14]:
random.seed(42)
lgb_models={}
for fold in range(5):
    print(f'===================================={fold+1}============================================')
    train_idx, valid_idx = folds[fold]
    X_train1, X_valid1, y_train1, y_valid1 = X_train.iloc[train_idx].values, X_train.iloc[valid_idx].values,\
                                         train_org['credit'][train_idx].values, train_org['credit'][valid_idx].values 
    lgb = LGBMClassifier(**lgbm_params, device = 'gpu')
    lgb.fit(X_train1, y_train1, 
            eval_set=[(X_train1, y_train1), (X_valid1, y_valid1)], 
            early_stopping_rounds=30,
           verbose=100)
    lgb_models[fold]=lgb
    print(f'================================================================================\n\n')

====================================1============================================


C:\Users\hwangtaegyun\anaconda3\lib\site-packages\lightgbm\sklearn.py:726: UserWarning: 'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. Pass 'early_stopping()' callback via 'callbacks' argument instead.
  _log_warning("'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. "
C:\Users\hwangtaegyun\anaconda3\lib\site-packages\lightgbm\sklearn.py:736: UserWarning: 'verbose' argument is deprecated and will be removed in a future release of LightGBM. Pass 'log_evaluation()' callback via 'callbacks' argument instead.
  _log_warning("'verbose' argument is deprecated and will be removed in a future release of LightGBM. "


[100]	training's multi_logloss: 0.61829	valid_1's multi_logloss: 0.741902
[200]	training's multi_logloss: 0.525841	valid_1's multi_logloss: 0.725769


====================================2============================================


C:\Users\hwangtaegyun\anaconda3\lib\site-packages\lightgbm\sklearn.py:726: UserWarning: 'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. Pass 'early_stopping()' callback via 'callbacks' argument instead.
  _log_warning("'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. "
C:\Users\hwangtaegyun\anaconda3\lib\site-packages\lightgbm\sklearn.py:736: UserWarning: 'verbose' argument is deprecated and will be removed in a future release of LightGBM. Pass 'log_evaluation()' callback via 'callbacks' argument instead.
  _log_warning("'verbose' argument is deprecated and will be removed in a future release of LightGBM. "


[100]	training's multi_logloss: 0.61698	valid_1's multi_logloss: 0.761524
[200]	training's multi_logloss: 0.523759	valid_1's multi_logloss: 0.748392


====================================3============================================


...

[100]	training's multi_logloss: 0.61849	valid_1's multi_logloss: 0.753444
[200]	training's multi_logloss: 0.525719	valid_1's multi_logloss: 0.743863


====================================4============================================


C:\Users\hwangtaegyun\anaconda3\lib\site-packages\lightgbm\sklearn.py:726: UserWarning: 'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. Pass 'early_stopping()' callback via 'callbacks' argument instead.
  _log_warning("'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. "
C:\Users\hwangtaegyun\anaconda3\lib\site-packages\lightgbm\sklearn.py:736: UserWarning: 'verbose' argument is deprecated and will be removed in a future release of LightGBM. Pass 'log_evaluation()' callback via 'callbacks' argument instead.
  _log_warning("'verbose' argument is deprecated and will be removed in a future release of LightGBM. "


[100]	training's multi_logloss: 0.616407	valid_1's multi_logloss: 0.747679
[200]	training's multi_logloss: 0.525538	valid_1's multi_logloss: 0.736646


====================================5============================================


C:\Users\hwangtaegyun\anaconda3\lib\site-packages\lightgbm\sklearn.py:726: UserWarning: 'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. Pass 'early_stopping()' callback via 'callbacks' argument instead.
  _log_warning("'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. "
C:\Users\hwangtaegyun\anaconda3\lib\site-packages\lightgbm\sklearn.py:736: UserWarning: 'verbose' argument is deprecated and will be removed in a future release of LightGBM. Pass 'log_evaluation()' callback via 'callbacks' argument instead.
  _log_warning("'verbose' argument is deprecated and will be removed in a future release of LightGBM. "


[100]	training's multi_logloss: 0.61699	valid_1's multi_logloss: 0.749648
[200]	training's multi_logloss: 0.522657	valid_1's multi_logloss: 0.739704




In [15]:
submit = pd.read_csv('../open/sample_submission.csv')

In [16]:
submit.iloc[:,1:]=0
for fold in range(5):
    submit.iloc[:,1:] += lgb_models[fold].predict_proba(X_test)/5

In [17]:
submit.to_csv('lgbm_test_submit_ensemble_no_fs.csv', index=False)

In [18]:
submit.head()

,index,0,1,2
0,26457,0.075494,0.121780,0.802726
1,26458,0.160599,0.149236,0.690165
2,26459,0.033276,0.074790,0.891933
3,26460,0.104767,0.137955,0.757278
4,26461,0.086604,0.185994,0.727403
